In [1]:
from rsna_retro.imports import *
from rsna_retro.metadata import *
from rsna_retro.preprocess import *
from rsna_retro.train import *
from rsna_retro.train3d import *

Loading imports


In [2]:
torch.cuda.set_device(2)

In [3]:
dls_feat = get_3d_dls_feat(Meta.df_comb, path=path_feat_384avg, bs=32)

## Model

In [4]:
xb, yb = dls_feat.one_batch(); xb.shape

torch.Size([32, 60, 512, 8, 8])

In [6]:
class NeuralNet(nn.Module):
    def __init__(self, n_classes=6, embed_size=1024, LSTM_UNITS=1024, DO = 0.3):
        super(NeuralNet, self).__init__()
        
#         self.embedding_dropout = SpatialDropout(0.0) #DO)
        self.flat = nn.Sequential(AdaptiveConcatPool2d(), Flatten())
        self.hook = ReshapeBodyHook(self.flat)
        
        self.lstm1 = nn.LSTM(embed_size, LSTM_UNITS, bidirectional=True, batch_first=True)
        self.lstm2 = nn.LSTM(LSTM_UNITS * 2, LSTM_UNITS, bidirectional=True, batch_first=True)

        self.linear1 = nn.Linear(LSTM_UNITS*2, LSTM_UNITS*2)
        self.linear2 = nn.Linear(LSTM_UNITS*2, LSTM_UNITS*2)

        self.linear = nn.Linear(LSTM_UNITS*2, n_classes)

    def forward(self, x):
        h_embedding = self.flat(x)
        h_embadd = torch.cat((h_embedding, h_embedding), -1)
        
        h_lstm1, _ = self.lstm1(h_embedding)
        h_lstm2, _ = self.lstm2(h_lstm1)
        
        h_conc_linear1  = F.relu(self.linear1(h_lstm1))
        h_conc_linear2  = F.relu(self.linear2(h_lstm2))
        
#         print([x.shape for x in [h_lstm1, h_lstm2, h_conc_linear1, h_conc_linear2, h_embadd]])
        
        hidden = h_lstm1 + h_lstm2 + h_conc_linear1 + h_conc_linear2 + h_embadd

        output = self.linear(hidden)
        
        return output

In [8]:
m = NeuralNet()
name = 'train3d_baseline_feat_lstm_2ndplace'
learn = get_learner(dls_feat, m, name=name)
learn.add_cb(DePadLoss())

In [9]:
# learn.summary()

## Training

In [10]:
learn.lr_find()

ValueError: too many values to unpack (expected 3)

In [9]:
do_fit(learn, 10, 1e-3)
learn.save(f'runs/{name}-1')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.054112,0.068720,0.977991,0.957986,01:25
1,0.053138,0.083151,0.972912,0.950538,01:24
2,0.051664,0.070177,0.977155,0.959221,01:20
3,0.048969,0.070529,0.977832,0.958712,01:19
4,0.052044,0.070904,0.977355,0.955452,01:20
5,0.049691,0.072793,0.977688,0.957642,01:20
6,0.049509,0.066312,0.978764,0.959246,01:20
7,0.048504,0.065887,0.979067,0.959616,01:20
8,0.048873,0.066936,0.979014,0.959666,01:31
9,0.046559,0.066987,0.978982,0.959386,01:24


## Testing

In [10]:
learn.dls = get_3d_dls_feat(Meta.df_tst, path=path_feat_tst_384avg, bs=32, test=True)

In [11]:
sub_fn = f'subm/{name}'
learn.load(f'runs/{name}-1')

In [12]:
preds,targs = learn.get_preds()
preds.shape, preds.min(), preds.max()

(torch.Size([121232, 6]), tensor(3.8507e-09), tensor(1.0000))

In [13]:
pred_csv = submission(Meta.df_tst, preds, fn=sub_fn)

In [14]:
api.competition_submit(f'{sub_fn}.csv', name, 'rsna-intracranial-hemorrhage-detection')

100%|██████████| 26.0M/26.0M [00:32<00:00, 846kB/s] 


Successfully submitted to RSNA Intracranial Hemorrhage Detection

In [19]:
api.competitions_submissions_list('rsna-intracranial-hemorrhage-detection')[0]

{'ref': 14320325,
 'totalBytes': 27277209,
 'date': '2020-01-29T08:20:57.827Z',
 'description': 'train3d_baseline_feat_lstm_2ndplace',
 'errorDescription': None,
 'fileName': 'train3d_baseline_feat_lstm_2ndplace.csv',
 'publicScore': '1.16287',
 'privateScore': '0.05369',
 'status': 'complete',
 'submittedBy': 'Andrew Shaw',
 'submittedByRef': 'bearpelican',
 'teamName': 'Andrew Shaw',
 'type': 'standard',
 'url': 'https://www.kaggle.com/submissions/14320325/14320325.raw'}